<a href="https://colab.research.google.com/github/krishnaselvam007/Guvi-assignment/blob/main/Task7_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You are provided with a dataset from USA Forensic Science Service
which has description of 6 types of glass; defined in terms of their
oxide content (i.e. Na, Fe, K, etc). Your task is to use K-Nearest
Neighbor (KNN) classifier to classify the glasses.
The original dataset is available at
(https://archive.ics.uci.edu/ml/datasets/glass+identification). For
detailed description on the attributes of the dataset,

#Data Cleaning - Exploratory Data Analysis

In [1]:
import pandas as pd
dataset = pd.read_csv('glass.csv') #read the dataset

In [2]:
print(dataset.shape)
dataset.head()

(214, 11)


,ID,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,2,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,3,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,4,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,5,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [3]:
dataset.describe()

,ID,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
count,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000
mean,107.500000,1.518365,13.407850,2.684533,1.444907,72.650935,0.497056,8.956963,0.175047,0.057009,2.780374
std,61.920648,0.003037,0.816604,1.442408,0.499270,0.774546,0.652192,1.423153,0.497219,0.097439,2.103739
min,1.000000,1.511150,10.730000,0.000000,0.290000,69.810000,0.000000,5.430000,0.000000,0.000000,1.000000
25%,54.250000,1.516522,12.907500,2.115000,1.190000,72.280000,0.122500,8.240000,0.000000,0.000000,1.000000
50%,107.500000,1.517680,13.300000,3.480000,1.360000,72.790000,0.555000,8.600000,0.000000,0.000000,2.000000
75%,160.750000,1.519157,13.825000,3.600000,1.630000,73.087500,0.610000,9.172500,0.000000,0.100000,3.000000
max,214.000000,1.533930,17.380000,4.490000,3.500000,75.410000,6.210000,16.190000,3.150000,0.510000,7.000000


In [4]:
dataset.isnull().sum()

ID      0
RI      0
Na      0
Mg      0
Al      0
Si      0
K       0
Ca      0
Ba      0
Fe      0
Type    0
dtype: int64

There is null data

In [5]:
dataset.dtypes

ID        int64
RI      float64
Na      float64
Mg      float64
Al      float64
Si      float64
K       float64
Ca      float64
Ba      float64
Fe      float64
Type      int64
dtype: object

All the datas are number, no need of encode

ID column is not required, we drop that

In [6]:
dataset = dataset.drop(columns=['ID'])
dataset

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0,1
...,...,...,...,...,...,...,...,...,...,...
209,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0,7
210,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0,7
211,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0,7
212,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0,7


In [7]:
dataset.Type.unique()

array([1, 2, 3, 5, 6, 7])

Following data cleaning, two Scikit-Learn KNN models should be created
for two different distance metrics: Square Euclidean and Manhattan
distance. The performance of the two models using different distance
metrics should be compared in terms of accuracy to the test data and
Scikit-Learn Classification Report.

**Splitting the dataset into the Training set and Test set**

In [8]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.plotting import plot_decision_regions
from sklearn.model_selection import cross_val_score # fn req for cross validation


In [16]:
from sklearn.decomposition import PCA

def knn_no_plot(data, k, p):
    x = data[['RI','Mg','Al','Si','K','Ca','Ba','Fe']].values # independent features #'RI','Mg','Al','Si','K','Ca','Ba','Fe'
    y = data['Type'].astype(int).values # y -> target value
    clf = KNeighborsClassifier(n_neighbors=k, p=p) #initialise the model with neighbours as k - hyper parameter p=1 - manhattan p=2 - Euclioden
    clf.fit(x, y) # train the model
    print("K : ", k, " Train Accuracy : ", clf.score(x,y), " Val Accuracy : ", 
          np.mean(cross_val_score(clf, x, y, cv=10)))  # test the model and it computes the accuracy (train data accuracy)



In [18]:
for i in [1,2,3,4,5,20,30,40,80]:
  #print(i)
  knn_no_plot(dataset,i, 1) # Manhattan Distance

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_se

K :  1  Train Accuracy :  1.0  Val Accuracy :  0.6357142857142857
K :  2  Train Accuracy :  0.8504672897196262  Val Accuracy :  0.6259740259740261
K :  3  Train Accuracy :  0.8551401869158879  Val Accuracy :  0.6497835497835498
K :  4  Train Accuracy :  0.8177570093457944  Val Accuracy :  0.6454545454545455
K :  5  Train Accuracy :  0.794392523364486  Val Accuracy :  0.6502164502164502
K :  20  Train Accuracy :  0.6822429906542056  Val Accuracy :  0.6413419913419913
K :  30  Train Accuracy :  0.6588785046728972  Val Accuracy :  0.6086580086580087
K :  40  Train Accuracy :  0.6261682242990654  Val Accuracy :  0.598917748917749
K :  80  Train Accuracy :  0.49065420560747663  Val Accuracy :  0.41147186147186143


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  UserWarning,


Following data cleaning, two Scikit-Learn KNN models should be created
for two different distance metrics: **Square Euclidean and Manhattan
distance**. The performance of the two models using different distance
metrics should be compared in terms of accuracy to the test data and
Scikit-Learn Classification Report.


In [17]:
for i in [1,2,3,4,5,20,30,40,80]:
  knn_no_plot(dataset,i, 2) # Euclidean Distance

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  UserWarning,


K :  1  Train Accuracy :  1.0  Val Accuracy :  0.6361471861471861
K :  2  Train Accuracy :  0.8364485981308412  Val Accuracy :  0.6313852813852814
K :  3  Train Accuracy :  0.8411214953271028  Val Accuracy :  0.65
K :  4  Train Accuracy :  0.780373831775701  Val Accuracy :  0.6274891774891775


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_se

K :  5  Train Accuracy :  0.7570093457943925  Val Accuracy :  0.6365800865800866
K :  20  Train Accuracy :  0.6588785046728972  Val Accuracy :  0.6367965367965368
K :  30  Train Accuracy :  0.6495327102803738  Val Accuracy :  0.637012987012987
K :  40  Train Accuracy :  0.6261682242990654  Val Accuracy :  0.6038961038961039
K :  80  Train Accuracy :  0.5747663551401869  Val Accuracy :  0.454112554112554


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  UserWarning,


Therefore best k value is 3 in **Euclidean** distance and 5 in **manhattan** distance